In [1]:
!pip install llama-index-llms-groq
%pip install llama-index-readers-file pymupdf
%pip install llama-index-embeddings-huggingface
%pip install llama-index-vector-stores-chroma
!pip install llama-index
!pip install llama-index-readers-json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from llama_index.llms.groq import Groq
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from getpass import getpass

# parameter setting
llm_model = 'llama3-70b-8192'
embed_model = "BAAI/bge-small-en"
file_path = '/content/sem-tab/' # now only with one file:r1_sample_metadata.jsonl
collection_name = 'sem-tab'

# enter your api key
key = getpass()
llm = Groq(model= llm_model, api_key=key)

# create client and a new collection
chroma_client = chromadb.EphemeralClient()
# clear past collection
chroma_client.delete_collection(collection_name)
# create new collection
chroma_collection = chroma_client.create_collection(collection_name)

# define embedding function
embed_model = HuggingFaceEmbedding(model_name=embed_model)

# load documents from a specific path(file or folders)
documents = SimpleDirectoryReader(file_path).load_data()

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

··········


In [5]:
# Query Data
query_engine = index.as_query_engine(llm)
response = query_engine.query("How many table in the file? List all tables and their column names")
display(Markdown(f"{response}"))

There are 7 tables in the file. Here is the list of tables and their column names:

1. Film - ["Fans' Rank", "Title", "Year", "Director(s)", "Overall Rank"]
2. Museum - ["RANG", "Museum", "Stadt", "Facebook-Fans"]
3. Film - ["Rank", "Title", "Year", "Director(s)", "Overall Rank"]
4. Mountain - ["Everest", "8.848", "Nepal/Tibet", "Asia"]
5. Film - ["Fans' Rank", "Title", "Year", "Director(s)", "Overall Rank"]
6. Bird - ["Common Name", "Genus", "Species", "IUCN"]
7. Airport - ["Code", "Airport Name", "City", "State"]
8. VideoGame - ["Year", "Game", "Genre", "Platform(s)", "Developer(s)"]
Note that there are three instances of the "Film" table, but they are considered separate tables in this context.